### Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display

import os
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import pandas as pd

import scipy.fft
from ipywidgets import interact
import ipywidgets as widgets

import h5py

import torch # downgraded to 1.10
import torchvision
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.utils.data
import torch.optim as optim
import torchaudio.transforms as T

import scipy.io

from scipy.signal import resample 

from torch.autograd import Variable

#import tensorflow as tf

import matplotlib.pyplot as plt

In [4]:
# Custom imports (code wrote by us)
from helpers import read_simulated_data, read_experimental_data, get_mel, resample_signal

In [5]:
%load_ext autoreload
%autoreload 2

### Import data

In [9]:
# Experimental files
#files = ['0_0BRB.mat']
         #,'0_1BRB.mat','0_2BRB.mat','0_3BRB.mat','0_0BRB.mat',
         #'25_0BRB.mat','25_1BRB.mat','25_2BRB.mat','25_3BRB.mat',
         #'50_0BRB.mat','50_1BRB.mat','50_2BRB.mat','50_3BRB.mat',
         #'75_0BRB.mat','75_1BRB.mat','75_2BRB.mat','75_3BRB.mat',
         #'100_0BRB.mat','100_1BRB.mat','100_2BRB.mat','100_3BRB.mat']

In [8]:
simulated_files = ['100_3BRB_Simu.mat']
experimental_files = ['100_3BRB.mat']
data_dir = 'data'

dataset = {}
for file in simulated_files:
    name, data_variable, time_variable = read_simulated_data(file, data_dir, verbose=False)
    dataset[name] = (data_variable, time_variable)
    
for file in experimental_files:
    name, data_variable, time_variable = read_experimental_data(file, data_dir, verbose=False)
    dataset[name] = (data_variable, time_variable)

In [21]:
import json 
import torch
from env import AttrDict
from models import Generator, MultiPeriodDiscriminator, MultiScaleDiscriminator

# Problem: we don't know which discriminator weights we got, 
# as there are two models and a single set of weights!!!

def load_model(
    model_to_load='generator', 
    model_dir='weights', 
    generator_name='g_02500000', 
    discriminator_name='do_02500000', 
    device='cpu'
):
    if model_to_load == 'generator':
        model_name = generator_name
    elif model_to_load=='discriminator':
        model_name = discriminator_name
    else:
        raise NotImplementedError
        
    model_file = os.path.join(model_dir, model_name)
    with open(os.path.join(model_dir,'config.json')) as f:
        _data = f.read()

    global h
    json_config = json.loads(_data)
    h = AttrDict(json_config) 
    
    # Init model
    if model_to_load == 'generator':
        generator = Generator(h).to(device)
        # Load checkpoint from file to the model weights
        state_dict = torch.load(model_file, map_location=device)
        generator.load_state_dict(state_dict['generator'])
        return generator
    else:
        mpd = MultiPeriodDiscriminator().to(device)
        msd = MultiScaleDiscriminator().to(device)
        # Load checkpoint from file to the model weights
        state_dict = torch.load(model_file, map_location=device) # for both models at the same time
        # dict_keys(['mpd', 'msd', 'optim_g', 'optim_d', 'steps', 'epoch'])
        mpd.load_state_dict(state_dict['mpd'])
        msd.load_state_dict(state_dict['msd'])
        return mpd, msd
    


In [20]:
generator = load_model(model_to_load='generator')

In [22]:
discriminators = load_model(model_to_load='discriminator')
mpd, msd = discriminators

state_dict.keys() dict_keys(['mpd', 'msd', 'optim_g', 'optim_d', 'steps', 'epoch'])


In [23]:
mpd

MultiPeriodDiscriminator(
  (discriminators): ModuleList(
    (0): DiscriminatorP(
      (convs): ModuleList(
        (0): Conv2d(1, 32, kernel_size=(5, 1), stride=(3, 1), padding=(2, 0))
        (1): Conv2d(32, 128, kernel_size=(5, 1), stride=(3, 1), padding=(2, 0))
        (2): Conv2d(128, 512, kernel_size=(5, 1), stride=(3, 1), padding=(2, 0))
        (3): Conv2d(512, 1024, kernel_size=(5, 1), stride=(3, 1), padding=(2, 0))
        (4): Conv2d(1024, 1024, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
      )
      (conv_post): Conv2d(1024, 1, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
    )
    (1): DiscriminatorP(
      (convs): ModuleList(
        (0): Conv2d(1, 32, kernel_size=(5, 1), stride=(3, 1), padding=(2, 0))
        (1): Conv2d(32, 128, kernel_size=(5, 1), stride=(3, 1), padding=(2, 0))
        (2): Conv2d(128, 512, kernel_size=(5, 1), stride=(3, 1), padding=(2, 0))
        (3): Conv2d(512, 1024, kernel_size=(5, 1), stride=(3, 1), padding=(2, 0))
        (4): 

In [24]:
msd

MultiScaleDiscriminator(
  (discriminators): ModuleList(
    (0): DiscriminatorS(
      (convs): ModuleList(
        (0): Conv1d(1, 128, kernel_size=(15,), stride=(1,), padding=(7,))
        (1): Conv1d(128, 128, kernel_size=(41,), stride=(2,), padding=(20,), groups=4)
        (2): Conv1d(128, 256, kernel_size=(41,), stride=(2,), padding=(20,), groups=16)
        (3): Conv1d(256, 512, kernel_size=(41,), stride=(4,), padding=(20,), groups=16)
        (4): Conv1d(512, 1024, kernel_size=(41,), stride=(4,), padding=(20,), groups=16)
        (5): Conv1d(1024, 1024, kernel_size=(41,), stride=(1,), padding=(20,), groups=16)
        (6): Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
      )
      (conv_post): Conv1d(1024, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (1): DiscriminatorS(
      (convs): ModuleList(
        (0): Conv1d(1, 128, kernel_size=(15,), stride=(1,), padding=(7,))
        (1): Conv1d(128, 128, kernel_size=(41,), stride=(2,), padding=(20,), g

In [11]:
def process_sample():
    pass